<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/Fade_Tester_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colormath
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from PIL import Image
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
import matplotlib.patches as patches
from google.colab import files

# Upload the CSV file containing LAB color dataset
print("Please upload your LAB color dataset CSV file.")
uploaded_files = files.upload()
csv_filename = next(iter(uploaded_files))

# Load the dataset
dataset = pd.read_csv(csv_filename)
lab_values = dataset[['L', 'A', 'B']].values
lab_tree = KDTree(lab_values)

# Step 2: Load and display the original image
print("Please upload the image file you want to analyze.")
uploaded_images = files.upload()
image_filename = next(iter(uploaded_images))
image = Image.open(image_filename).convert('RGB')
plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')
plt.title('Original Image')
plt.show()

# Convert image to numpy array and calculate average color
image_array = np.array(image)
average_color = image_array.mean(axis=(0, 1)) / 255.0

# Display average color patch
average_color_patch = np.ones((100, 100, 3)) * average_color
plt.figure(figsize=(2, 2))
plt.imshow(average_color_patch)
plt.axis('off')
plt.title('Average Color Extracted')
plt.show()

# Convert the average color to LAB
rgb_color = sRGBColor(*average_color)
lab_color = convert_color(rgb_color, LabColor)
L, A, B = lab_color.lab_l, lab_color.lab_a, lab_color.lab_b

# Find closest colors in the dataset
distances, indices = lab_tree.query([L, A, B], k=5)
closest_colors = dataset.iloc[indices]

# Display closest colors and their names
fig, axs = plt.subplots(1, 5, figsize=(15, 3))
for i, (index, dist) in enumerate(zip(indices, distances)):
    closest_color = dataset.iloc[index]
    lab_color_closest = LabColor(closest_color['L'], closest_color['A'], closest_color['B'])
    rgb_color_closest = convert_color(lab_color_closest, sRGBColor)

    rgb_values_closest = [max(0, min(1, val)) for val in [rgb_color_closest.rgb_r, rgb_color_closest.rgb_g, rgb_color_closest.rgb_b]]
    color_patch = np.ones((100, 100, 3)) * rgb_values_closest
    axs[i].imshow(color_patch)
    axs[i].axis('off')
    axs[i].set_title(f"{closest_color['Color Name']}\nDist: {dist:.2f}")

plt.show()

### New Feature 1: Pixel-by-Pixel Color Difference Mapping ###

def color_difference_map(image1, image2):
    """
    Create a pixel-by-pixel color difference map between two images in LAB color space.
    """
    image1_array = np.array(image1)
    image2_array = np.array(image2)

    diff_image = np.zeros_like(image1_array)

    for i in range(image1_array.shape[0]):
        for j in range(image1_array.shape[1]):
            # Convert RGB to LAB
            rgb1 = sRGBColor(*(image1_array[i, j] / 255.0))
            lab1 = convert_color(rgb1, LabColor)

            rgb2 = sRGBColor(*(image2_array[i, j] / 255.0))
            lab2 = convert_color(rgb2, LabColor)

            # Calculate ∆E (color difference in LAB space)
            delta_e = np.sqrt((lab1.lab_l - lab2.lab_l)**2 +
                              (lab1.lab_a - lab2.lab_a)**2 +
                              (lab1.lab_b - lab2.lab_b)**2)

            diff_image[i, j] = delta_e

    return diff_image

# Step 3: Compare pixel-by-pixel color difference between two images (original vs faded)
print("Please upload the faded image for comparison.")
uploaded_faded_images = files.upload()
faded_image_filename = next(iter(uploaded_faded_images))
faded_image = Image.open(faded_image_filename).convert('RGB')

color_diff = color_difference_map(image, faded_image)
plt.imshow(color_diff, cmap='hot')
plt.colorbar()
plt.title("Pixel-by-Pixel Color Difference Map (∆E)")
plt.show()

### New Feature 2: Block-by-Block Color Comparison ###

def block_comparison(image1, image2, block_size=50):
    """
    Compare two images block by block to visualize changes.
    """
    h, w, _ = image1.shape
    result_image = np.zeros_like(image1)

    for y in range(0, h, block_size):
        for x in range(0, w, block_size):
            block1 = image1[y:y+block_size, x:x+block_size]
            block2 = image2[y:y+block_size, x:x+block_size]

            # Compute average color difference for the block
            avg_diff = np.mean(np.abs(block1.astype(float) - block2.astype(float)))
            result_image[y:y+block_size, x:x+block_size] = avg_diff

    plt.imshow(result_image, cmap='coolwarm')
    plt.title(f"Block-by-Block Comparison (Block Size: {block_size}px)")
    plt.show()

block_comparison(np.array(image), np.array(faded_image), block_size=50)

### Updated Feature 3: Histogram Analysis (RGB and LAB) ###

def plot_histograms(image1, image2):
    """
    Plot RGB and LAB histograms for two images.
    """
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))

    # Flatten the RGB values for both images and plot histograms
    image1_flat = image1.reshape(-1, 3)
    image2_flat = image2.reshape(-1, 3)

    # RGB Histogram
    axs[0, 0].hist(image1_flat[:, 0], bins=256, color='r', alpha=0.5, label='Red (Original)')
    axs[0, 0].hist(image2_flat[:, 0], bins=256, color='b', alpha=0.5, label='Red (Faded)')
    axs[0, 0].set_title('Red Channel Histogram (Original vs Faded)')
    axs[0, 0].legend()

    axs[0, 1].hist(image1_flat[:, 1], bins=256, color='g', alpha=0.5, label='Green (Original)')
    axs[0, 1].hist(image2_flat[:, 1], bins=256, color='y', alpha=0.5, label='Green (Faded)')
    axs[0, 1].set_title('Green Channel Histogram (Original vs Faded)')
    axs[0, 1].legend()

    axs[1, 0].hist(image1_flat[:, 2], bins=256, color='b', alpha=0.5, label='Blue (Original)')
    axs[1, 0].hist(image2_flat[:, 2], bins=256, color='cyan', alpha=0.5, label='Blue (Faded)')
    axs[1, 0].set_title('Blue Channel Histogram (Original vs Faded)')
    axs[1, 0].legend()

    # LAB Histogram
    image1_lab = np.array([convert_color(sRGBColor(*pixel/255.0), LabColor).get_value_tuple() for pixel in image1_flat])
    image2_lab = np.array([convert_color(sRGBColor(*pixel/255.0), LabColor).get_value_tuple() for pixel in image2_flat])

    axs[1, 1].hist(image1_lab[:, 0], bins=256, color='magenta', alpha=0.5, label='L* (Original)')
    axs[1, 1].hist(image2_lab[:, 0], bins=256, color='orange', alpha=0.5, label='L* (Faded)')
    axs[1, 1].set_title('LAB L* Histogram (Original vs Faded)')
    axs[1, 1].legend()

    plt.tight_layout()
    plt.show()

# Call the updated function
plot_histograms(np.array(image), np.array(faded_image))

### New Feature 4: Line-by-Line Color Extraction ###

def plot_line_by_line(image1, image2, line):
    """
    Compare color along a specific line in two images.
    """
    line_colors1 = image1[line, :, :]
    line_colors2 = image2[line, :, :]

    plt.plot(np.arange(line_colors1.shape[0]), line_colors1.mean(axis=1), label='Original')
    plt.plot(np.arange(line_colors2.shape[0]), line_colors2.mean(axis=1), label='Faded')
    plt.legend()
    plt.title(f'Line-by-Line Color Comparison (Line: {line})')
    plt.show()

plot_line_by_line(np.array(image), np.array(faded_image), line=50)

### New Feature 5: Color Fading Simulation ###

def simulate_fading(image1, image2, alpha=0.5):
    """
    Simulate color fading between two images by blending them.
    """
    blended_image = Image.blend(image1, image2, alpha)
    plt.imshow(blended_image)
    plt.title(f"Simulated Fading (Alpha: {alpha})")
    plt.show()

simulate_fading(image, faded_image, alpha=0.5)